In [14]:
import pandas as pd
import mysql.connector

Cargamos los dataframes que necesitamos

In [15]:
df_ataques = pd.read_csv("../data/06-tiburon_4.csv", index_col = 0)
df_ataques.head(2)

,year,type,country,activity,age,species,month,fatal,gender,id_country
0,2018,Boating,usa,Paddling,57.0,White,Jun,N,F,0
1,2018,Unprovoked,usa,Walking,15.0,Bull,May,N,M,0


In [16]:
df_climas = pd.read_csv("../data/01-datos_clima.csv", index_col= 0)
df_climas.head(2)

,timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,country,latitud,longitud,id_country
0,3,7,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 5}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 150, 'speed':...",16,6,4,1024,none,0,0,110,2,usa,39.78373,-100.445882,0
1,6,8,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 6}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 265, 'speed':...",15,6,6,1025,none,0,0,220,2,usa,39.78373,-100.445882,0


In [17]:
df_paises = pd.read_csv("../data/00-paises.csv", index_col= 0)
df_paises.head(2)

,id_country,country
0,0,usa
1,1,brazil


# Pair Programming ETL Carga I
Igual que en el ejercicio de pair programming de ETL Transformación II, tendréis que crear una clase que nos permita cargar los datos en SQL I.

Creamos la clase

In [18]:
class Subir_SQL:
    def __init__(self, nombre_bbdd, contraseña):
        # nuestra clase va tener atributos, nombre de la BBDD y la contraseña 
        self.nombre_bbdd = nombre_bbdd
        self.contraseña = contraseña
    
    # Creamos la función para crear la base de datos
    def crear_bbdd(self):

        mydb = mysql.connector.connect(
        host="localhost",
        user="root",
        password= f'{self.contraseña}' 
        )
        print("Conexión realizada con éxito")
        
        mycursor = mydb.cursor()

        try:
            mycursor.execute(f"CREATE DATABASE IF NOT EXISTS {self.nombre_bbdd};")
            print(mycursor)
        except mysql.connector.Error as err:
            print(err)
            print("Error Code:", err.errno)
            print("SQLSTATE", err.sqlstate)
            print("Message", err.msg)


    # Creamos una funcion para crear nuestras tablas
    def crear_tabla(self, query):
    
    # nos conectamos con el servidor usando el conector de sql
        cnx = mysql.connector.connect(user='root', password=f"{self.contraseña}",
                                        host='127.0.0.1', database=f"{self.nombre_bbdd}")
        # iniciamos el cursor
        mycursor = cnx.cursor()
        
        # intentamos hacer la query
        try: 
            mycursor.execute(query)
            cnx.commit() 
        # en caso de que podamos ejecutar la query devuelvenos un error para saber en que nos estamos equivocando
        except mysql.connector.Error as err:
            print(err)
            print("Error Code:", err.errno)
            print("SQLSTATE", err.sqlstate)
            print("Message", err.msg)

    # Creamos una función para insertar los datos
    def insercion_datos(self, query):
        cnx = mysql.connector.connect(user='root', password=f"{self.contraseña}",
                                    host='127.0.0.1', database=f"{self.nombre_bbdd}")
        mycursor = cnx.cursor()
        
        try: 
            # Le añadimos un escape para poder subir los datos a pesar de tener una foreign key
            mycursor.execute("SET foreign_key_checks = 0;")
            mycursor.execute(query)
            cnx.commit()
            mycursor.execute("SET foreign_key_checks = 1;")

        except mysql.connector.Error as err:
            print(err)
            print("Error Code:", err.errno)
            print("SQLSTATE", err.sqlstate)
            print("Message", err.msg)
        

In [19]:
# Llamamos a la clase
ata = Subir_SQL("tiburones2", "AlumnaAdalab")

In [20]:
# Creamos la base de datos
ata.crear_bbdd()

Conexión realizada con éxito
MySQLCursor: CREATE DATABASE IF NOT EXISTS tiburones2..


In [21]:
# Query para crear la tabla paises
tabla_paises = '''
CREATE TABLE IF NOT EXISTS `paises` (
  `country` VARCHAR (45) NOT NULL,
  `id_country` INT NOT NULL,
  PRIMARY KEY (`id_country`))
  ;'''

In [22]:
# Creamos la query para crear la tabla clima, con una foreign key a paises
tabla_clima = '''
CREATE TABLE IF NOT EXISTS `clima` (
  `id_clima` INT NOT NULL AUTO_INCREMENT,
  `timepoint` INT NOT NULL,
  `cloudcover` INT NOT NULL,
  `highcloud` INT NOT NULL,
  `midcloud` INT NOT NULL,
  `lowcloud` INT NOT NULL,
  `rh_profile` VARCHAR (45) NOT NULL,
  `wind_profile` VARCHAR (45) NOT NULL,
  `temp2m` INT NOT NULL,
  `lifted_index` INT NOT NULL,
  `rh2m` INT NOT NULL,
  `msl_pressure` INT NOT NULL,
  `prec_type` VARCHAR (45) NOT NULL,
  `prec_amount` INT NOT NULL,
  `snow_depth` INT NOT NULL,
  `wind10m.direction` INT NOT NULL,
  `wind10m.speed` INT NOT NULL,
  `country` VARCHAR (45) NOT NULL,
  `latitud` DECIMAL NOT NULL,
  `longitud` DECIMAL NOT NULL,
  `id_country` INT NOT NULL,
  PRIMARY KEY (`id_clima`),
  INDEX `fk_ataques_paises1_idx` (`id_country` ASC) ,
  CONSTRAINT `fk_clima_paises` 
    FOREIGN KEY (`id_country`) 
    REFERENCES `paises`(`id_country`))
  ENGINE = InnoDB
  ;'''

In [23]:
# Creamos la query para crear la tabla ataques, con una foreign key a paises
tabla_ataques = """
CREATE TABLE IF NOT EXISTS `ataques` (
    `id_ataques` INT NOT NULL AUTO_INCREMENT PRIMARY KEY,
    `year` INT NOT NULL,
    `type` VARCHAR(200) NOT NULL,
    `country` VARCHAR(45) NOT NULL,
    `activity` VARCHAR(200) NOT NULL,
    `age` INT NOT NULL,
    `species` VARCHAR(200) NOT NULL,
    `month` VARCHAR(200) NOT NULL,
    `fatal` VARCHAR(200) NOT NULL,
    `gender` VARCHAR(200) NOT NULL,
    `id_country` INT NOT NULL,
    INDEX `fk_ataques_paises1_idx` (`id_country` ASC) ,
    CONSTRAINT `fk_ataques_paises` 
        FOREIGN KEY (`id_country`) 
        REFERENCES `paises`(`id_country`))
    ENGINE = InnoDB
    ;"""

In [24]:
# Creamos las tablas
ata.crear_tabla(tabla_paises)
ata.crear_tabla(tabla_clima)
ata.crear_tabla(tabla_ataques)

In [25]:
df_ataques["activity"] = df_ataques["activity"].replace(r'"', '', regex= True)

In [26]:
# Iteramos por las filas del df ataques para ir insertando los datos en nuestra BBDD
for indice, fila in df_ataques.iterrows(): # iteramos por el dataframe.
    
    # definimos nuestra query, igual que si lo hicieramos en workbench. ⚠️ Como estamos definiendo nuestra query en varias líneas usamos las triples comillas
    # lo valores que introduciremos serán los del dataframe que estamos iterando, por lo que usaremos los formats de los strings. 
    
    query_ataques = f"""
            INSERT INTO ataques (year, type, country, activity, age, species, month, fatal, gender, id_country)
            VALUES ( "{fila['year']}", "{fila['type']}", "{fila['country']}", "{fila['activity']}", "{fila['age']}", "{fila['species']}", "{fila['month']}", "{fila['fatal']}", "{fila['gender']}", "{fila['id_country']}");
            """
    # una vez definida la query llamamos a la función que nos inserta los datos. 
    ata.insercion_datos(query_ataques)